<h1><center>Tarea 1: Esquemas de Decodificacion Instantanea - INFO145</center></h1>


# 1. Importacion de T[1..N] y Librerias

In [326]:
#importacion libreria

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [327]:
def importarTexto(x):
    file = open(x,'r')
    data=file.readlines() 
    file.close() 
    data_str = []
    for i in range(len(data)):
        for j in range (len(data[i])):
            data_str += data[i][j]
    return data_str

In [328]:
data_prueba = importarTexto('prueba.txt')
data_dna = importarTexto('dna.1MB')
data_english = importarTexto('english.1MB')

# 2. Obtencion del alfabeto del texto

In [329]:
def no_esta(array,b):
    verificador = True
    for i in range(len(array)):
        if array[i] == b :
            verificador = False
    return verificador

In [330]:
def alfabeto(x):
    alfabeto = []
    for i in range(len(x)):
        if no_esta(alfabeto,x[i]):
            alfabeto +=  x[i]
    return alfabeto


In [331]:
alfabeto_dna = alfabeto(data_dna)
alfabeto_english = alfabeto(data_english)
alfabeto_prueba = alfabeto(data_prueba)

# 3. Implementacion Algoritmo Shannon

Los pasos para desarrollar este algoritmo son los siguientes: 
* **(1) Para una secuencia de símbolos, se calcula la correspondiente lista de frecuencias de aparición de los símbolos**
* **(2) Se ordena la lista de símbolos según su frecuencia en orden decreciente**
* **(3) Se divide la lista en dos partes, de forma que la suma total de frecuencias de las 2 mitades sean lo más cercana posible**
* **(4)  A la mitad superior de la lista se le asigna el dígito binario 1, y a la mitad inferior se le asigna el dígito binario 0. Esto significa que los códigos de los símbolos en la primera mitad empezarán todos con 1 y los códigos en la segunda mitad empezarán todos con 0**
* **(5) Recursivamente se aplica el mismo procedimiento a cada una de las dos mitades, se subdivide en grupos y se agregan bits a las códigos hasta que cada grupo conste de un único símbolo**

In [351]:
def probabilidad(simbolo,simbolo_totales,sec): # funcion para calcular probabilidades
    contador = 0
    contador = contador + sec.count(simbolo)
    return contador/simbolo_totales
def get_indice(x): # obtiene el indice donde se va a dividir la lista
    c= 0
    q= len(x)-1
    a = x.Probabilidad[c]
    b = x.Probabilidad[q]
    while (c+1) != q and c!=q:
        if b<=a:
            b += x.Probabilidad[(q-1)]
            q -= 1
        if a<b :
            a += x.Probabilidad[(c+1)]
            c+= 1
    if c== q :
        c -= 1
    return c,q 
def div_list(x): # divide las lista en 2 
        fin_lista_1 = get_indice(x)[0]
        comienzo_lista_2 = get_indice(x)[1]
        div_1 = x[0:fin_lista_1+1]
        div_2 = x[comienzo_lista_2:]
        return div_1, div_2
def code(x):
    if len(x)>=2:
        table_1 = div_list(x)[0] #obtiene la lista de la mitad superior
        table_2 = div_list(x)[1] #obtiene la lista de la mitad inferior
        table_1.Code += '1'
        code(table_1)  # recursivamente codifica la lista de la mitad superior
        table_2.Code +='0'
        code(table_2)  # recursivamente codifica la lista de la mitad superior
        return x

def shannon(x, sec): # x corresponde al alfabeto , sec a secuencia
#........................................................................................................................
#......................Primero se calculara la frecuencia de los simbolos del alfabeto....................................
#.........................................................................................................................
    simbolos_totales = len(sec) #numero total de simbolos de la secuencia
    probs = [] # se crea un arreglo con  sus probabilidades 
    for i in range(len(x)):
        probs += [probabilidad(x[i],simbolos_totales,sec)]
#..........................................................................................................................
#......................Se Creara un dataframe con los simbolos y sus probabilidades........................................
#..........................................................................................................................
    df = pd.DataFrame({'Simbolo': x,'Probabilidad': probs, 'Code': ''})
    df.index = df['Simbolo']
    df.drop(['Simbolo'], axis= 1 , inplace= True)
#...........................................................................................................................
#.............................Ordenamiento de las probabilidades(Descendente)...............................................
#...........................................................................................................................
    df.sort_values(by=['Probabilidad'], ascending = False, inplace = True)
#............................................................................................................................
#..........................................Obtencion del alfabeto de codificacion............................................
#............................................................................................................................
    df = code(df)
    return df
#............................................................................................................................
#...............................................Codificacion.................................................................
#............................................................................................................................
def codificacionShannon(x,sec,largo):# x corresponde al alfabeto , sec a secuencia, largo = tramo de la secuencia a codificar
    df = shannon(x,sec)
    codificacion = ''
    for i in range(largo): # largo tiene que se >0
        codificacion += df[df.index == sec[i]].Code[0]
    return codificacion
    
        

In [389]:
codificacionShannon(alfabeto_english,data_english,50) # codificacion 

'0011100111001110011100111000011000101011101101111110101111000000011111011111001001100101101111001000001100101001000001101001011110010001110001010001111110000010011010011000011001001010011110000011101010010010010111010101110110001100001110011100111'

# 4. Implementacion Algoritmo Decodificacion Shannon

# 5. Implementacion Algoritmo Huffman Canonico

# 6. Implementacion Algoritmo Decodificacion Huffman Canonico